# Reihenfolge auf Maschinen

In [1]:
import pandas as pd

pd.set_option('display.max_colwidth', 93)

In [ ]:
# pip install Levenshtein

In [2]:
import utils.sequence_tester as seq_tester

In [3]:
def get_schedule_filename(prefix: str = "", day: int = 0, data_txt: str = "schedule", suffix: str = "") -> str:
    file_template = "data/{prefix}_{data}_{day:02d}{suffix}.csv"
    if suffix:
        suffix = f"_{suffix}"
    return file_template.format(prefix=prefix,data=data_txt,day=day, suffix=suffix)

In [4]:
get_schedule_filename("06", day=0, suffix="init")

'data/06_schedule_00_init.csv'

In [5]:
df_plan_init =  pd.read_csv(get_schedule_filename("05", day=1, data_txt="plan", suffix="init"))
df_plan_init = df_plan_init.sort_values(by=["Arrival", "Start"]).reset_index(drop=True)
df_plan_init

,Job,Operation,Machine,Arrival,Deadline,Start,Processing Time,End,Lateness,Tardiness,Earliness
0,Job_000,0,M0,0,2015,1440,29,1469,-546,0,546
1,Job_000,1,M1,0,2015,1486,78,1564,-451,0,451
2,Job_000,2,M2,0,2015,1628,9,1637,-378,0,378
3,Job_000,3,M3,0,2015,1637,36,1673,-342,0,342
4,Job_000,4,M4,0,2015,1703,49,1752,-263,0,263
...,...,...,...,...,...,...,...,...,...,...,...
185,Job_018,5,M9,1398,3839,3006,76,3082,-757,0,757
186,Job_018,6,M5,1398,3839,3082,47,3129,-710,0,710
187,Job_018,7,M3,1398,3839,3129,52,3181,-658,0,658
188,Job_018,8,M4,1398,3839,3181,90,3271,-568,0,568


In [6]:
df_plan_init[df_plan_init.Job == "Job_018"]

,Job,Operation,Machine,Arrival,Deadline,Start,Processing Time,End,Lateness,Tardiness,Earliness
180,Job_018,0,M1,1398,3839,2324,85,2409,-1430,0,1430
181,Job_018,1,M0,1398,3839,2409,13,2422,-1417,0,1417
182,Job_018,2,M2,1398,3839,2504,61,2565,-1274,0,1274
183,Job_018,3,M6,1398,3839,2768,7,2775,-1064,0,1064
184,Job_018,4,M8,1398,3839,2942,64,3006,-833,0,833
185,Job_018,5,M9,1398,3839,3006,76,3082,-757,0,757
186,Job_018,6,M5,1398,3839,3082,47,3129,-710,0,710
187,Job_018,7,M3,1398,3839,3129,52,3181,-658,0,658
188,Job_018,8,M4,1398,3839,3181,90,3271,-568,0,568
189,Job_018,9,M7,1398,3839,3794,45,3839,0,0,0


## Mit "Deviation Penalty"

### Tag 2

In [7]:
day_numb = 2

df_plan_temp = pd.read_csv(get_schedule_filename("06_devpen", day=day_numb))
df_plan_temp = df_plan_temp.sort_values(by=["Arrival", "Start"]).reset_index(drop=True)
df_plan_temp 

df_plan_devpen_01 = df_plan_temp 
df_plan_devpen_01

,Job,Operation,Arrival,Deadline,Machine,Start,Processing Time,End,Lateness,Tardiness,Earliness
0,Job_003,9,203,2981,M5,2926,43,2969,-12,0,12
1,Job_008,9,731,3339,M8,3230,74,3304,-35,0,35
2,Job_009,9,764,3205,M7,3162,45,3207,2,2,0
3,Job_010,9,872,3310,M3,3231,79,3310,0,0,0
4,Job_011,7,924,3531,M7,2890,89,2979,-552,0,552
...,...,...,...,...,...,...,...,...,...,...,...
215,Job_037,5,2838,5279,M9,4021,76,4097,-1182,0,1182
216,Job_037,6,2838,5279,M5,4097,47,4144,-1135,0,1135
217,Job_037,7,2838,5279,M3,4144,52,4196,-1083,0,1083
218,Job_037,8,2838,5279,M4,4260,90,4350,-929,0,929


#### Vergleich (Kendall Tau)

In [8]:
df_compare_01 = seq_tester.analyze_machine_sequences(df_plan_init, df_plan_devpen_01, 1440*day_numb)
df_compare_01

,original_sequence,revised_sequence_raw,revised_sequence,changed,kendall_tau
Machine,,,,,
M3,"[Job_018, Job_010, Job_014]","[Job_025, Job_019, Job_023, Job_018, Job_010, Job_014, Job_020, Job_021, Job_035, Job_024...","[Job_018, Job_010, Job_014]",False,1.0
M4,"[Job_011, Job_018, Job_016]","[Job_013, Job_011, Job_019, Job_018, Job_020, Job_023, Job_031, Job_029, Job_016, Job_025...","[Job_011, Job_018, Job_016]",False,1.0
M5,"[Job_003, Job_018, Job_014]","[Job_003, Job_016, Job_023, Job_018, Job_019, Job_032, Job_029, Job_024, Job_025, Job_020...","[Job_003, Job_018, Job_014]",False,1.0
M6,"[Job_016, Job_015]","[Job_013, Job_014, Job_016, Job_015, Job_019, Job_025, Job_020, Job_023, Job_028, Job_021...","[Job_016, Job_015]",False,1.0
M7,"[Job_016, Job_014, Job_009, Job_013, Job_018]","[Job_011, Job_016, Job_014, Job_009, Job_019, Job_013, Job_023, Job_025, Job_020, Job_021...","[Job_016, Job_014, Job_009, Job_013, Job_018]",False,1.0
M8,"[Job_018, Job_014, Job_008, Job_012, Job_011]","[Job_018, Job_014, Job_008, Job_012, Job_019, Job_023, Job_011, Job_025, Job_021, Job_024...","[Job_018, Job_014, Job_008, Job_012, Job_011]",False,1.0
M9,"[Job_018, Job_016, Job_017, Job_014]","[Job_018, Job_016, Job_017, Job_020, Job_014, Job_019, Job_025, Job_023, Job_031, Job_024...","[Job_018, Job_016, Job_017, Job_014]",False,1.0


### Tag 3

In [9]:
day_numb = 3

df_plan_temp = pd.read_csv(get_schedule_filename("06_devpen", day=day_numb))
df_plan_temp = df_plan_temp .sort_values(by=["Arrival", "Start"]).reset_index(drop=True)
df_plan_temp 

df_plan_devpen_02 = df_plan_temp 
df_plan_devpen_02

,Job,Operation,Arrival,Deadline,Machine,Start,Processing Time,End,Lateness,Tardiness,Earliness
0,Job_022,7,1643,4421,M3,4401,98,4499,78,78,0
1,Job_022,8,1643,4421,M9,4499,22,4521,100,100,0
2,Job_022,9,1643,4421,M5,4521,43,4564,143,143,0
3,Job_026,6,1944,4382,M8,4366,19,4385,3,3,0
4,Job_026,7,1944,4382,M9,4438,48,4486,104,104,0
...,...,...,...,...,...,...,...,...,...,...,...
237,Job_056,5,4278,6719,M9,5623,76,5699,-1020,0,1020
238,Job_056,6,4278,6719,M5,5699,47,5746,-973,0,973
239,Job_056,7,4278,6719,M3,5822,52,5874,-845,0,845
240,Job_056,8,4278,6719,M4,6064,90,6154,-565,0,565


In [10]:
og = df_plan_devpen_01
new = df_plan_devpen_02

df_compare_02 = seq_tester.analyze_machine_sequences(og, new, 1440*day_numb)
df_compare_02

,original_sequence,revised_sequence_raw,revised_sequence,changed,kendall_tau
Machine,,,,,
M0,[Job_032],"[Job_038, Job_053, Job_055, Job_042, Job_039, Job_032, Job_052, Job_056, Job_050, Job_048...",[Job_032],False,NaN
M3,"[Job_028, Job_029, Job_033]","[Job_028, Job_022, Job_038, Job_026, Job_042, Job_029, Job_044, Job_039, Job_037, Job_043...","[Job_028, Job_029, Job_033]",False,1.000000
M4,"[Job_034, Job_028, Job_032, Job_027, Job_030, Job_035]","[Job_034, Job_028, Job_032, Job_038, Job_027, Job_039, Job_042, Job_050, Job_030, Job_052...","[Job_034, Job_028, Job_032, Job_027, Job_030, Job_035]",False,1.000000
M5,"[Job_022, Job_033]","[Job_035, Job_022, Job_042, Job_038, Job_043, Job_044, Job_053, Job_039, Job_045, Job_051...","[Job_022, Job_033]",False,1.000000
M6,"[Job_030, Job_032, Job_034, Job_035]","[Job_036, Job_030, Job_029, Job_032, Job_033, Job_034, Job_035, Job_038, Job_044, Job_056...","[Job_030, Job_032, Job_034, Job_035]",False,1.000000
M7,"[Job_027, Job_034, Job_029, Job_028, Job_031, Job_030, Job_032, Job_033, Job_037]","[Job_036, Job_034, Job_026, Job_027, Job_035, Job_029, Job_028, Job_031, Job_030, Job_032...","[Job_034, Job_027, Job_029, Job_028, Job_031, Job_030, Job_032, Job_033, Job_037]",True,0.944444
M8,"[Job_036, Job_029, Job_034, Job_027, Job_031, Job_030]","[Job_026, Job_034, Job_036, Job_029, Job_033, Job_027, Job_031, Job_042, Job_030, Job_038...","[Job_034, Job_036, Job_029, Job_027, Job_031, Job_030]",True,0.733333
M9,"[Job_032, Job_029, Job_034, Job_036, Job_035, Job_033]","[Job_032, Job_026, Job_022, Job_034, Job_029, Job_036, Job_035, Job_039, Job_044, Job_050...","[Job_032, Job_034, Job_029, Job_036, Job_035, Job_033]",True,0.866667


### Tag 3

In [11]:
day_numb = 3

df_plan_temp = pd.read_csv(get_schedule_filename("06_devpen", day=day_numb))
df_plan_temp = df_plan_temp .sort_values(by=["Arrival", "Start"]).reset_index(drop=True)
df_plan_temp 

df_plan_devpen_03 = df_plan_temp 

In [12]:
og = df_plan_devpen_02
new = df_plan_devpen_03

df_compare_03 = seq_tester.analyze_machine_sequences(og, new, 1440*day_numb)
df_compare_03

,original_sequence,revised_sequence_raw,revised_sequence,changed,kendall_tau
Machine,,,,,
M0,"[Job_038, Job_053, Job_055, Job_042, Job_039, Job_032, Job_052, Job_056, Job_050, Job_048...","[Job_038, Job_053, Job_055, Job_042, Job_039, Job_032, Job_052, Job_056, Job_050, Job_048...","[Job_038, Job_053, Job_055, Job_042, Job_039, Job_032, Job_052, Job_056, Job_050, Job_048...",False,1.0
M1,"[Job_052, Job_038, Job_056, Job_042, Job_044, Job_041, Job_055, Job_043, Job_040, Job_053...","[Job_052, Job_038, Job_056, Job_042, Job_044, Job_041, Job_055, Job_043, Job_040, Job_053...","[Job_052, Job_038, Job_056, Job_042, Job_044, Job_041, Job_055, Job_043, Job_040, Job_053...",False,1.0
M2,"[Job_042, Job_053, Job_045, Job_052, Job_038, Job_048, Job_039, Job_043, Job_050, Job_055...","[Job_042, Job_053, Job_045, Job_052, Job_038, Job_048, Job_039, Job_043, Job_050, Job_055...","[Job_042, Job_053, Job_045, Job_052, Job_038, Job_048, Job_039, Job_043, Job_050, Job_055...",False,1.0
M3,"[Job_028, Job_022, Job_038, Job_026, Job_042, Job_029, Job_044, Job_039, Job_037, Job_043...","[Job_028, Job_022, Job_038, Job_026, Job_042, Job_029, Job_044, Job_039, Job_037, Job_043...","[Job_028, Job_022, Job_038, Job_026, Job_042, Job_029, Job_044, Job_039, Job_037, Job_043...",False,1.0
M4,"[Job_034, Job_028, Job_032, Job_038, Job_027, Job_039, Job_042, Job_050, Job_030, Job_052...","[Job_034, Job_028, Job_032, Job_038, Job_027, Job_039, Job_042, Job_050, Job_030, Job_052...","[Job_034, Job_028, Job_032, Job_038, Job_027, Job_039, Job_042, Job_050, Job_030, Job_052...",False,1.0
M5,"[Job_035, Job_022, Job_042, Job_038, Job_043, Job_044, Job_053, Job_039, Job_045, Job_051...","[Job_035, Job_022, Job_042, Job_038, Job_043, Job_044, Job_053, Job_039, Job_045, Job_051...","[Job_035, Job_022, Job_042, Job_038, Job_043, Job_044, Job_053, Job_039, Job_045, Job_051...",False,1.0
M6,"[Job_036, Job_030, Job_029, Job_032, Job_033, Job_034, Job_035, Job_038, Job_044, Job_056...","[Job_036, Job_030, Job_029, Job_032, Job_033, Job_034, Job_035, Job_038, Job_044, Job_056...","[Job_036, Job_030, Job_029, Job_032, Job_033, Job_034, Job_035, Job_038, Job_044, Job_056...",False,1.0
M7,"[Job_036, Job_034, Job_026, Job_027, Job_035, Job_029, Job_028, Job_031, Job_030, Job_032...","[Job_036, Job_034, Job_026, Job_027, Job_035, Job_029, Job_028, Job_031, Job_030, Job_032...","[Job_036, Job_034, Job_026, Job_027, Job_035, Job_029, Job_028, Job_031, Job_030, Job_032...",False,1.0
M8,"[Job_026, Job_034, Job_036, Job_029, Job_033, Job_027, Job_031, Job_042, Job_030, Job_038...","[Job_026, Job_034, Job_036, Job_029, Job_033, Job_027, Job_031, Job_042, Job_030, Job_038...","[Job_026, Job_034, Job_036, Job_029, Job_033, Job_027, Job_031, Job_042, Job_030, Job_038...",False,1.0


## Ohne "Deviation Penalty"

### Tag 2

In [13]:
day_numb = 2

df_plan_temp = pd.read_csv(get_schedule_filename("06_simple", day=day_numb))
df_plan_temp = df_plan_temp .sort_values(by=["Arrival", "Start"]).reset_index(drop=True)
df_plan_temp 

df_plan_simple_01 = df_plan_temp 
df_plan_simple_01

,Job,Operation,Arrival,Deadline,Machine,Start,Processing Time,End,Lateness,Tardiness,Earliness
0,Job_003,9,203,2981,M5,2938,43,2981,0,0,0
1,Job_008,9,731,3339,M8,3230,74,3304,-35,0,35
2,Job_009,9,764,3205,M7,3160,45,3205,0,0,0
3,Job_010,9,872,3310,M3,3231,79,3310,0,0,0
4,Job_011,7,924,3531,M7,2890,89,2979,-552,0,552
...,...,...,...,...,...,...,...,...,...,...,...
215,Job_037,5,2838,5279,M9,4246,76,4322,-957,0,957
216,Job_037,6,2838,5279,M5,4322,47,4369,-910,0,910
217,Job_037,7,2838,5279,M3,4382,52,4434,-845,0,845
218,Job_037,8,2838,5279,M4,4434,90,4524,-755,0,755


In [14]:
og = df_plan_init
new = df_plan_simple_01

df_compare_simple_01 = seq_tester.analyze_machine_sequences(og, new, 1440*day_numb)
df_compare_simple_01

,original_sequence,revised_sequence_raw,revised_sequence,changed,kendall_tau
Machine,,,,,
M3,"[Job_018, Job_010, Job_014]","[Job_025, Job_019, Job_021, Job_010, Job_023, Job_020, Job_018, Job_024, Job_014, Job_027...","[Job_010, Job_018, Job_014]",True,0.333333
M4,"[Job_011, Job_018, Job_016]","[Job_013, Job_011, Job_020, Job_019, Job_023, Job_026, Job_018, Job_022, Job_016, Job_031...","[Job_011, Job_018, Job_016]",False,1.000000
M5,"[Job_003, Job_018, Job_014]","[Job_003, Job_016, Job_023, Job_024, Job_019, Job_018, Job_025, Job_026, Job_034, Job_029...","[Job_003, Job_018, Job_014]",False,1.000000
M6,"[Job_016, Job_015]","[Job_013, Job_014, Job_016, Job_015, Job_025, Job_019, Job_026, Job_020, Job_028, Job_023...","[Job_016, Job_015]",False,1.000000
M7,"[Job_016, Job_014, Job_009, Job_013, Job_018]","[Job_011, Job_016, Job_009, Job_014, Job_019, Job_013, Job_023, Job_020, Job_025, Job_021...","[Job_016, Job_009, Job_014, Job_013, Job_018]",True,0.800000
M8,"[Job_018, Job_014, Job_008, Job_012, Job_011]","[Job_018, Job_014, Job_008, Job_012, Job_019, Job_023, Job_011, Job_025, Job_021, Job_024...","[Job_018, Job_014, Job_008, Job_012, Job_011]",False,1.000000
M9,"[Job_018, Job_016, Job_017, Job_014]","[Job_016, Job_018, Job_020, Job_017, Job_025, Job_019, Job_023, Job_014, Job_024, Job_021...","[Job_016, Job_018, Job_017, Job_014]",True,0.666667


### Tag 3

In [15]:
day_numb = 3

df_plan_temp = pd.read_csv(get_schedule_filename("06_simple", day=day_numb))
df_plan_temp = df_plan_temp .sort_values(by=["Arrival", "Start"]).reset_index(drop=True)
df_plan_temp 

df_plan_simple_02 = df_plan_temp 
df_plan_simple_02

,Job,Operation,Arrival,Deadline,Machine,Start,Processing Time,End,Lateness,Tardiness,Earliness
0,Job_022,9,1643,4421,M5,4378,43,4421,0,0,0
1,Job_027,8,2171,4779,M4,4443,26,4469,-310,0,310
2,Job_027,9,2171,4779,M8,4670,74,4744,-35,0,35
3,Job_028,8,2204,4645,M4,4353,90,4443,-202,0,202
4,Job_028,9,2204,4645,M7,4600,45,4645,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
219,Job_056,5,4278,6719,M9,5163,76,5239,-1480,0,1480
220,Job_056,6,4278,6719,M5,5695,47,5742,-977,0,977
221,Job_056,7,4278,6719,M3,5996,52,6048,-671,0,671
222,Job_056,8,4278,6719,M4,6048,90,6138,-581,0,581


In [16]:
og = df_plan_simple_01
new = df_plan_simple_02

df_compare_simple_02 = seq_tester.analyze_machine_sequences(og, new, 1440*day_numb)
df_compare_simple_02

,original_sequence,revised_sequence_raw,revised_sequence,changed,kendall_tau
Machine,,,,,
M3,"[Job_037, Job_033, Job_029]","[Job_037, Job_038, Job_029, Job_042, Job_043, Job_044, Job_039, Job_054, Job_040, Job_050...","[Job_037, Job_029, Job_033]",True,0.333333
M4,"[Job_032, Job_030, Job_037, Job_035]","[Job_028, Job_027, Job_032, Job_030, Job_038, Job_052, Job_042, Job_039, Job_050, Job_037...","[Job_032, Job_030, Job_037, Job_035]",False,1.000000
M5,"[Job_037, Job_022, Job_035, Job_033]","[Job_022, Job_035, Job_037, Job_043, Job_038, Job_042, Job_044, Job_053, Job_039, Job_051...","[Job_022, Job_035, Job_037, Job_033]",True,0.333333
M6,"[Job_034, Job_035]","[Job_032, Job_033, Job_034, Job_038, Job_052, Job_035, Job_056, Job_044, Job_039, Job_042...","[Job_034, Job_035]",False,1.000000
M7,"[Job_033, Job_035, Job_028, Job_032, Job_037]","[Job_030, Job_033, Job_035, Job_028, Job_031, Job_038, Job_032, Job_042, Job_052, Job_039...","[Job_033, Job_035, Job_028, Job_032, Job_037]",False,1.000000
M8,"[Job_027, Job_031, Job_030]","[Job_035, Job_033, Job_036, Job_027, Job_031, Job_052, Job_038, Job_042, Job_030, Job_056...","[Job_027, Job_031, Job_030]",False,1.000000
M9,"[Job_033, Job_036, Job_035]","[Job_034, Job_036, Job_038, Job_039, Job_035, Job_042, Job_050, Job_044, Job_043, Job_056...","[Job_036, Job_035, Job_033]",True,-0.333333


## Levenstein

In [17]:
df_compare_simple_02

,original_sequence,revised_sequence_raw,revised_sequence,changed,kendall_tau
Machine,,,,,
M3,"[Job_037, Job_033, Job_029]","[Job_037, Job_038, Job_029, Job_042, Job_043, Job_044, Job_039, Job_054, Job_040, Job_050...","[Job_037, Job_029, Job_033]",True,0.333333
M4,"[Job_032, Job_030, Job_037, Job_035]","[Job_028, Job_027, Job_032, Job_030, Job_038, Job_052, Job_042, Job_039, Job_050, Job_037...","[Job_032, Job_030, Job_037, Job_035]",False,1.000000
M5,"[Job_037, Job_022, Job_035, Job_033]","[Job_022, Job_035, Job_037, Job_043, Job_038, Job_042, Job_044, Job_053, Job_039, Job_051...","[Job_022, Job_035, Job_037, Job_033]",True,0.333333
M6,"[Job_034, Job_035]","[Job_032, Job_033, Job_034, Job_038, Job_052, Job_035, Job_056, Job_044, Job_039, Job_042...","[Job_034, Job_035]",False,1.000000
M7,"[Job_033, Job_035, Job_028, Job_032, Job_037]","[Job_030, Job_033, Job_035, Job_028, Job_031, Job_038, Job_032, Job_042, Job_052, Job_039...","[Job_033, Job_035, Job_028, Job_032, Job_037]",False,1.000000
M8,"[Job_027, Job_031, Job_030]","[Job_035, Job_033, Job_036, Job_027, Job_031, Job_052, Job_038, Job_042, Job_030, Job_056...","[Job_027, Job_031, Job_030]",False,1.000000
M9,"[Job_033, Job_036, Job_035]","[Job_034, Job_036, Job_038, Job_039, Job_035, Job_042, Job_050, Job_044, Job_043, Job_056...","[Job_036, Job_035, Job_033]",True,-0.333333


### Ohne DevPen

In [27]:
og = df_compare_simple_02.loc["M5", "original_sequence"]
og

['Job_037', 'Job_022', 'Job_035', 'Job_033']

In [28]:
rev = df_compare_simple_02.loc["M5", "revised_sequence"]
rev

['Job_022', 'Job_035', 'Job_037', 'Job_033']

In [29]:
from Levenshtein import distance

og_str = ",".join(og)
rev_str = ",".join(rev)
lev_dist = distance(og_str, rev_str)
print(lev_dist)


5


### Mit DevPen

In [37]:
og = df_compare_02.loc["M7", "original_sequence"]
og

['Job_027',
 'Job_034',
 'Job_029',
 'Job_028',
 'Job_031',
 'Job_030',
 'Job_032',
 'Job_033',
 'Job_037']

In [38]:
rev = df_compare_02.loc["M7", "revised_sequence"]
rev

['Job_034',
 'Job_027',
 'Job_029',
 'Job_028',
 'Job_031',
 'Job_030',
 'Job_032',
 'Job_033',
 'Job_037']

In [39]:
og_str = ",".join(og)
rev_str = ",".join(rev)
lev_dist = distance(og_str, rev_str)
print(lev_dist)


4


In [35]:
def spearman_footrule(seq1: list, seq2: list) -> int:
    """
    Berechnet Spearman's Fußregel-Distanz zwischen zwei Job-Sequenzen.
    Beide Sequenzen müssen die gleichen Elemente enthalten.
    """
    # Mapping von Job → Rang in beiden Sequenzen
    rank1 = {job: i for i, job in enumerate(seq1)}
    rank2 = {job: i for i, job in enumerate(seq2)}

    # Summe der absoluten Differenzen der Ränge
    return sum(abs(rank1[job] - rank2[job]) for job in seq1)


In [36]:
footrule = spearman_footrule(og, rev)
print(f"Spearman’s Fußregel für M7: {footrule}")

Spearman’s Fußregel für M7: 2
